In [1]:
import pandas as pd
import jieba

# 一、读取《复仇者联盟4》的好评，中评，差评评论数据

In [2]:
#好评
pos_com =  pd.read_csv('comments_sets/复仇者联盟4_好评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
# sep设置为文档内不包含的内容，否则出错
pos_com.columns = ['comments']

#中评
mid_com  = pd.read_csv('comments_sets/复仇者联盟4_中评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
mid_com.columns = ['comments']


#差评
neg_com = pd.read_csv('comments_sets/复仇者联盟4_差评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
neg_com.columns = ['comments']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


In [3]:
pos_com.head()

,comments
0,如果你不喜欢这部电影，说明他不是为你准备的，故事的终章是为读过故事的人准备的
1,我是一个90后，我曾经很羡慕“上一代人”：40年前的观众，他们的影院里有星战正传三部曲的落幕...
2,托尼说好要回归家庭、陪伴家人，可最终还是选择了重出江湖，因为责任和使命，因为“我是钢铁侠”。
3,美队曾经错过了和卡特的约会，错过陪伴卡特的70年，后来又出席卡特的葬礼，亲自送挚爱上路，感谢...
4,黑寡妇高尚、豪迈、英勇、无畏，和钢铁侠并列最伟大的复仇者。


In [4]:
comment_pos = pos_com['comments']
comment_mid = mid_com['comments']
comment_neg = neg_com['comments']
print('好评的评论数为：',comment_pos.shape)
print('中评的评论数为：',comment_mid.shape)
print('差评的评论数为：',comment_neg.shape)

好评的评论数为： (68163,)
中评的评论数为： (23484,)
差评的评论数为： (23490,)


# 二、去除重复词及水军评论

In [9]:
def condense_1(str):
	# 这里i代表每次处理的字符单位数，如i=1时处理“好好好好”的情况，i=2时处理“很好很好很好”的情况
	# i=1&i=2时用一种处理方式，即当重复数量>2时才进行压缩，因为出现“滔滔不绝”、“美的的确好”
	# 跟“容我思考思考”“这真的真的好看”等不好归为冗余的情况。但当出现3次及以上时基本就是冗余了。
	for i in [1, 2]:
		j = 0
		while j < len(str)-2*i:
			#判断重复了至少两次
			if str[j: j+i] == str[j+i: j+2*i] and str[j: j+i] == str[j+2*i: j+3*i]:
				k = j+2*i
				while k+i<len(str) and str[j: j+i]==str[k+i: k+2*i]:
					k += i
				str = str[: j+i] + str[k+i:]
			j += 1
		i += 1
	
	# i=3&i=4时用一种处理方式，当重复>1时就进行压缩，因为3个字以上时重复不再构成成语或其他常用语，
	# 基本上即使冗余了。因为大于五个字的重复比较少出现，为了减少算法复杂度可以只处理到i=4。
	for i in [3, 4, 5]:
		j = 0
		while j < len(str)-2*i:
			#判断重复了至少一次
			if str[j: j+i]==str[j+i: j+2*i]:
				k = j+i
				while k+i<len(str) and str[j: j+i]==str[k+i: k+2*i]:
					k += i
				str = str[: j+i] + str[k+i:]
			j += 1
		i += 1
	
	return str

In [10]:
#总评论的字符长度
print('好评的总字符长度为：',comment_pos.astype('str').apply(lambda x: len(x)).sum())
print('中评的总字符长度为：',comment_mid.astype('str').apply(lambda x: len(x)).sum())
print('差评的总字符长度为：',comment_neg.astype('str').apply(lambda x: len(x)).sum())

好评的总字符长度为： 5043387
中评的总字符长度为： 1814759
差评的总字符长度为： 1629345


In [11]:
data1_pos = comment_pos.astype('str').apply(lambda x: condense_1(x))  # 去除重复词
data1_mid = comment_mid.astype('str').apply(lambda x: condense_1(x))  # 去除重复词
data1_neg = comment_neg.astype('str').apply(lambda x: condense_1(x))  # 去除重复词

#去除重复词后的字符长度
print('去除重复词后的好评的总字符长度为：',data1_pos.apply(lambda x: len(x)).sum())
print('去除重复词后的中评的总字符长度为：',data1_mid.apply(lambda x: len(x)).sum())
print('去除重复词后的差评的总字符长度为：',data1_neg.apply(lambda x: len(x)).sum())


去除重复词后的好评的总字符长度为： 5014525
去除重复词后的中评的总字符长度为： 1802192
去除重复词后的差评的总字符长度为： 1621282


In [16]:
data2_pos = data1_pos.apply(lambda x: len(x))
data2_mid = data1_mid.apply(lambda x: len(x))
data2_neg = data1_neg.apply(lambda x: len(x))

data3_pos = pd.concat((data1_pos, data2_pos), axis = 1)  # 合并
data3_pos.columns = ['评论','长度']
data3_mid = pd.concat((data1_mid, data2_mid), axis = 1)  # 合并
data3_mid.columns = ['评论','长度']
data3_neg = pd.concat((data1_neg, data2_neg), axis = 1)  # 合并
data3_neg.columns = ['评论','长度']

data3_neg.head()

,评论,长度
0,一群好几百岁的人叽叽歪歪像一群青春期的小孩，三个小时的时长就靠没完没了的与过去的自己相遇+废...,61
1,将近300元买的杜比首映场，是真的不值。除了花里胡哨的特效外，一无是处，没有插科打诨和反高潮...,124
2,前20分钟真的以为罗素兄弟要捧出什么好菜，适量精准的文戏和反高潮的设置让我眼前一亮。然后电影...,350
3,不得不承认流量在这个时代主宰世界,16
4,三小时超长CP混剪，还是水准贼烂的那种，十年你就给我这个丢人的玩意作答卷？,37


In [17]:
data3_pos['长度'].value_counts().sort_index()[:10]  # 好评长度前十的数量统计

3      479
4      690
5      893
6      297
7      596
8     1494
9     1076
10    1580
11    2849
12    1376
Name: 长度, dtype: int64

In [19]:
data4_pos = data3_pos.loc[data3_pos['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('好评非水军评论',data4_pos.shape)

data4_mid = data3_mid.loc[data3_mid['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('中评非水军评论',data4_mid.shape)

data4_neg = data3_neg.loc[data3_neg['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('差评非水军评论',data4_neg.shape)

好评非水军评论 (66101,)
中评非水军评论 (23109,)
差评非水军评论 (22279,)


In [20]:
data4_pos.head()

0               如果你不喜欢这部电影，说明他不是为你准备的，故事的终章是为读过故事的人准备的
1    我是一个90后，我曾经很羡慕“上一代人”：40年前的观众，他们的影院里有星战正传三部曲的落幕...
2       托尼说好要回归家庭、陪伴家人，可最终还是选择了重出江湖，因为责任和使命，因为“我是钢铁侠”。
3    美队曾经错过了和卡特的约会，错过陪伴卡特的70年，后来又出席卡特的葬礼，亲自送挚爱上路，感谢...
4                        黑寡妇高尚、豪迈、英勇、无畏，和钢铁侠并列最伟大的复仇者。
Name: 评论, dtype: object

# 三、分词

In [21]:
import jieba
#example
list(jieba.cut('我爱北京天安门，天安门前国旗升'))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\p'ro\AppData\Local\Temp\jieba.cache
Loading model cost 0.771 seconds.
Prefix dict has been built succesfully.


['我', '爱', '北京', '天安门', '，', '天安门', '前', '国旗', '升']

In [22]:
#好评
data5_pos = data4_pos.apply(lambda x: list(jieba.cut(x)))  
#中评
data5_mid = data4_mid.apply(lambda x: list(jieba.cut(x))) 
#差评
data5_neg = data4_neg.apply(lambda x: list(jieba.cut(x)))

data5_pos.head()

0    [如果, 你, 不, 喜欢, 这部, 电影, ，, 说明, 他, 不是, 为, 你, 准备,...
1    [我, 是, 一个, 90, 后, ，, 我, 曾经, 很, 羡慕, “, 上, 一代人, ...
2    [托尼, 说好, 要, 回归, 家庭, 、, 陪伴, 家人, ，, 可, 最终, 还是, 选...
3    [美队, 曾经, 错过, 了, 和, 卡特, 的, 约会, ，, 错过, 陪伴, 卡特, 的...
4    [黑寡妇, 高尚, 、, 豪迈, 、, 英勇, 、, 无畏, ，, 和, 钢铁, 侠, 并列...
Name: 评论, dtype: object

# 四、去停用词

In [25]:
#读取网上下载的停用词表
stop = pd.read_csv('stoplist.txt', sep='aab', encoding = 'utf-8', header = None)
# sep设置为文档内不包含的内容，否则出错
stop = ['漫威','电影'] + [' ', ''] + list(stop[0]) 	#Pandas自动过滤了空格符，这里手动添加

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [27]:
data6_pos = data5_pos.apply(lambda x: [i for i in x if i not in stop])

data6_mid = data5_mid.apply(lambda x: [i for i in x if i not in stop])

data6_neg = data5_neg.apply(lambda x: [i for i in x if i not in stop])


In [28]:
print('去停用词后的好评字符总长:',data6_pos.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_pos.apply(lambda x: len(x)).sum() - data6_pos.apply(lambda x: len(x)).sum())

print('去停用词后的中评字符总长:',data6_mid.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_mid.apply(lambda x: len(x)).sum() - data6_mid.apply(lambda x: len(x)).sum())

print('去停用词后的差评字符总长:',data6_neg.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_neg.apply(lambda x: len(x)).sum() - data6_neg.apply(lambda x: len(x)).sum())

去停用词后的好评字符总长: 1353167 减少长度: 1753310
去停用词后的中评字符总长: 487446 减少长度: 626274
去停用词后的差评字符总长: 450838 减少长度: 548276


In [34]:
data6_pos.head()

0                             [喜欢, 这部, 故事, 终章, 读过, 故事]
1    [90, 羡慕, 一代人, 40, 年前, 观众, 影院, 里, 星战, 正传, 三部曲, ...
2    [托尼, 说好, 回归, 家庭, 陪伴, 家人, 最终, 选择, 重出江湖, 责任, 使命,...
3    [美队, 错过, 卡特, 约会, 错过, 陪伴, 卡特, 70, 年, 出席, 卡特, 葬礼...
4                [黑寡妇, 高尚, 豪迈, 英勇, 无畏, 钢铁, 侠, 并列, 复仇者]
Name: 评论, dtype: object

In [35]:
data6_mid.head()

0    [感情, 分, 合格, 水准, 通篇, 流水账, 观感, 第三部, 这类, 片, 无所谓, ...
1    [钢铁, 侠, 头号, 一哥, 支撑, 复联, 女拳, 队长, 角色, 定位, 彻头彻尾, ...
2    [三个, 小时, 剧情, 太, 拖沓, 成, 七龙珠, 寻宝, 堆砌, 拼凑, 剧情, 浪费...
3    [复联, 一场, 寓言故事, 黑色, 歌剧, 惊艳, 不为过, 所有人, 猜想, 复联, 何...
4    [抱歉, 谢幕, 实在, 吹, 出口, 剧情, 逻辑, bug, care, 人物, ooc...
Name: 评论, dtype: object

In [36]:
data6_neg.head()

0    [一群, 好几百, 岁, 叽叽, 歪歪, 一群, 青春期, 小孩, 三个, 小时, 时长, ...
1    [300, 杜比, 首映, 场, 真的, 不值, 花里胡哨, 特效, 外, 一无是处, 插科...
2    [前, 20, 分钟, 真的, 罗素, 兄弟, 捧, 好菜, 适量, 精准, 文戏, 反, ...
3                                   [承认, 流量, 时代, 主宰世界]
4    [小时, 超长, CP, 混剪, 水准, 贼, 烂, 那种, 十年, 丢人, 玩意, 作, 答卷]
Name: 评论, dtype: object

In [37]:
#保存好评，中评，差评的经过预处理的数据
data6_pos.to_csv('comments_sets/pre_pos.txt',encoding = 'utf-8', index = False, header = False)

data6_mid.to_csv('comments_sets/pre_mid.txt',encoding = 'utf-8', index = False, header = False)

data6_neg.to_csv('comments_sets/pre_neg.txt',encoding = 'utf-8', index = False, header = False)

In [ ]:
#end